In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import os
import random
import glob
import pickle
import tqdm
from models import *
import time

Using TensorFlow backend.


In [2]:
#Set Base directory as Capstone path
dir_path=os.getcwd()
try: 
    os.chdir(os.getcwd()[0:dir_path.index('Code')])
except Exception:
    print(Exception)


In [3]:
import inspect
source_DF = inspect.getsource(Model_List.CNN)

print(source_DF)

    def CNN(self):
        #frames_input = Input(shape=self.input_shape)
        model = Sequential()
        model.add(Conv2D(64, (1, 2), activation='relu', padding='same', input_shape=(10, 256, 38)))
        model.add(MaxPooling2D((1, 2)))
        model.add(Conv2D(64, (1, 2), activation='relu'))
        model.add(MaxPooling2D((1, 2)))
        model.add(Conv2D(128, (1, 2), activation='relu'))
        model.add(MaxPooling2D((1, 2)))
        model.add(Conv2D(128, (1, 2), activation='relu'))
        model.add(MaxPooling2D((1, 2)))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(1))
        #model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
        #CNN = Model(inputs=frames_input, outputs=output)
        return model



In [7]:
def generator(IDs, yields, batch_size, cutoff=None):
    import numpy as np
    import random
    
 # Create empty arrays to get batch of features and labels

    if cutoff != None:
        batch_features = np.zeros((batch_size, cutoff, 1, 256, 10))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('Data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('Data/PROCESSED_III/' + ID + '.npy')[:cutoff, :, :, :]
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
                    
            yield batch_features, batch_yields
                    
    else:
        batch_features = np.zeros((batch_size, 38, 1, 256, 10))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('Data/PROCESSED_III/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :, :] = np.load('Data/PROCESSED_III/' + ID + '.npy')
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
            yield batch_features, batch_yields
            
def generator_CNN(IDs, yields, batch_size, cutoff=None):
    import numpy as np
    import random
    
 # Create empty arrays to get batch of features and labels

    if cutoff != None:
        batch_features = np.zeros((batch_size, cutoff, 256, 10))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('Data/PROCESSED_IV/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :] = np.load('Data/PROCESSED_IV/' + ID + '.npy')[:cutoff, :, :]
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
                    
            yield batch_features, batch_yields
                    
    else:
        batch_features = np.zeros((batch_size, 38, 256, 10))
        batch_yields = np.zeros((batch_size))
        while True:
            for i in range(batch_size):
                # choose random index in features
                index = random.choice(range(len(IDs)))
                ID = IDs[index]
                if np.sum(np.isnan(np.load('Data/PROCESSED_IV/' + ID + '.npy'))) == 0:
                    batch_features[i, :, :, :] = np.load('Data/PROCESSED_IV/' + ID + '.npy')
                    #print('yes', ID)
                    batch_yields[i] = yields[ID]
                else:
                    print('no', ID)
            yield batch_features, batch_yields

# Model Training

In [10]:
start = time.time()

# training model on data of year 2010-2015 (6 years total)

#model_list = ['CNN_LSTM', 'SepCNN_LSTM', 'CONVLSTM']
model_list = ['CNN_AVG','CNN_AVG_small']

#Get Datasets
yields = pickle.load(open('Data/yields.p', 'rb'))
y = yields
print(len(yields['train']), len(yields['validation']))

# Early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

# generators
#training_generator = generator(list(y['train'].keys()), y['train'], 16)
#validation_generator = generator(list(y['validation'].keys()), y['validation'], 16)

for model_name in model_list:
    if model_name in model_list:
        training_generator = generator_CNN(list(y['train'].keys()), y['train'], 64)
        validation_generator = generator_CNN(list(y['validation'].keys()), y['validation'], 64)
    else:
        training_generator = generator(list(y['train'].keys()), y['train'], 64)
        validation_generator = generator(list(y['validation'].keys()), y['validation'], 64)
    rm = Model_List(model_name, 38, (1, 256, 10), print_model=True)   
    history=rm.model.fit_generator(training_generator, validation_data=validation_generator, \
                                   callbacks= callbacks_list, \
                               validation_steps=50, steps_per_epoch=100, nb_epoch=30, verbose=0)
    print (model_name)
    mae = history.history['mean_absolute_error']
    val_mae = history.history['val_mean_absolute_error']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(mae) + 1)
    plt.plot(epochs, mae, 'bo', label='Training mae')
    plt.plot(epochs, val_mae, 'b', label='Validation mae')
    plt.title('Training and validation MAE')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    rm.model.save(model_name)
end = time.time()
print(int(end - start)/60,' mins')

76 9
Loading Model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 10, 256, 64)       4928      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 128, 64)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 127, 64)       8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 63, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 62, 128)       16512     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 31, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 30, 128)       3

C:\Users\Vinayak\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Vinayak\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., callbacks=[<keras.ca..., validation_steps=84.5625, verbose=0, steps_per_epoch=5, epochs=50)`


Epoch 00025: early stopping
4.95  mins


In [13]:
%pwd

'C:\\Users\\Vinayak\\Downloads\\CBA\\Capstone'

In [16]:
import matplotlib.pyplot as plt

#rm = Model_List('No model', 38, (1, 256, 10), saved_model=model_name)
history = load_model('CNN')
#history=model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
#                               validation_steps=1353/16, samples_per_epoch=5, nb_epoch=50, verbose=0)
mae = history.history['mean_absolute_error']
val_mae = history.history['val_mean_absolute_error']
loss = history.history['loss']
#al_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

AttributeError: 'Sequential' object has no attribute 'history'

# Model Evaluation

In [12]:
start = time.time()

# Model Evaluation on the yields of paddy yields across India in year 2016
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']))
X_test, y_test = next(test_gen)
abs_error = np.empty(len(model_list))

for i, model_name in enumerate(model_list):
    rm = Model_List('No model', 38, (1, 256, 10), saved_model=model_name)
    abs_error[i] = rm.model.evaluate(X_test, y_test, batch_size=16)[0]
    print('For model {}, the test mean absolute error is {:.2f}.'.format(model_name, abs_error[i]))

best_model = model_list[np.argmin(abs_error)]
print('The best model is {}.'.format(best_model))
end = time.time()
print(int(end - start)/60,' mins')

Loading model CNN


ValueError: Error when checking input: expected conv2d_5_input to have 4 dimensions, but got array with shape (9, 38, 1, 256, 10)

# Optimization of batch size

In [ ]:
start = time.time()
####################

batch_size = [8, 32, 64]
abs_error_batch = np.empty(len(batch_size))

for i, size in enumerate(batch_size):
    # generators
    training_generator = generator(list(y['train'].keys()), y['train'], size)
    validation_generator = generator(list(y['validation'].keys()), y['validation'], size)

    rm = Model_List('None', 38, (1, 256, 10), saved_model=best_model)
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/size, samples_per_epoch=50, nb_epoch=10, verbose=0)

    abs_error_batch[i] = rm.model.evaluate(X_test, y_test, batch_size=size)[1]
    print('For batch {}, the test mean absolute error is {:.2f}.'.format(size, abs_error_batch[i]))
    
    rm.model.save(best_model + '_' + str(size))
print(int(time.time() - start)/60,' mins')

# Reducing the number of frames per year

In [ ]:
start = time.time()
########################
n_frames = [1, 5, 10, 15, 20, 25, 30, 35]
abs_error_frames = np.empty(len(n_frames))
i = 0

for frame in n_frames:
    # generators
    print('Doing {} frames per year'.format(frame))
    training_generator = generator(list(y['train'].keys()), y['train'], 16, cutoff=frame)
    validation_generator = generator(list(y['validation'].keys()), y['validation'], 16, cutoff=frame)
    
    test_gen = generator(list(yields['validation'].keys()), yields['validation'], len(yields['validation']), cutoff=frame)
    X_test, y_test = next(test_gen)

    rm = Model_List('CNN_LSTM', frame, (1, 256, 10))
    rm.model.fit_generator(training_generator, validation_data=validation_generator, callbacks=callbacks_list,\
                               validation_steps=1353/16, samples_per_epoch=50, nb_epoch=10, verbose=0)
    abs_error_frames[i] = rm.model.evaluate(X_test, y_test, batch_size=16)[1]
    
    rm.model.save('CNN_LSTM' + '_' + str(frame))
    print('For {} frames per year, the test mean absolute error is {:.2f}.'.format(frame, abs_error_frames[i]))
    i += 1
    pickle.dump( abs_error_frames, open( "abs_error_framess.p", "wb" ) )

######################## 
print(int(time.time() - start)/60,' mins')